In [4]:
def rank(similarity, q_pids, g_pids, topk=[1, 5, 10], get_mAP=True):
    max_rank = max(topk)
    if get_mAP:
        indices = torch.argsort(similarity, dim=1, descending=True)
    else:
        # acclerate sort with topk
        _, indices = torch.topk(
            similarity, k=max_rank, dim=1, largest=True, sorted=True
        )  # q * topk
    pred_labels = g_pids[indices]  # q * k
    matches = pred_labels.eq(q_pids.view(-1, 1))  # q * k

    all_cmc = matches[:, :max_rank].cumsum(1)
    all_cmc[all_cmc > 1] = 1
    all_cmc = all_cmc.float().mean(0) * 100
    all_cmc = all_cmc[topk - 1]

    if not get_mAP:
        return all_cmc, indices

    num_rel = matches.sum(1)  # q
    tmp_cmc = matches.cumsum(1)  # q * k
    tmp_cmc = [tmp_cmc[:, i] / (i + 1.0) for i in range(tmp_cmc.shape[1])]
    tmp_cmc = torch.stack(tmp_cmc, 1) * matches
    AP = tmp_cmc.sum(1) / num_rel  # q
    mAP = AP.mean() * 100
    return all_cmc, mAP, indices

In [5]:
import pickle as pkl
import numpy as np
import torch
topk = torch.tensor([1,5,10])

In [6]:
similarity = np.load("similarity.npy")
similarity_fine = np.load("similarity_fine.npy")
text_pid = np.load("text_pid.npy")
image_pid = np.load("image_pid.npy")

In [13]:
final_similarity = similarity + similarity_fine

In [14]:
t2i_cmc, _ = rank(torch.tensor(final_similarity), torch.tensor(text_pid), torch.tensor(image_pid), topk, get_mAP=False)
print(t2i_cmc)

tensor([60.7537, 79.4184, 86.6309])


In [9]:
weights = [i/10 for i in range(20)]
weights

[0.0,
 0.1,
 0.2,
 0.3,
 0.4,
 0.5,
 0.6,
 0.7,
 0.8,
 0.9,
 1.0,
 1.1,
 1.2,
 1.3,
 1.4,
 1.5,
 1.6,
 1.7,
 1.8,
 1.9]

In [15]:
for weight in weights:
    print(weight)
    final_similarity = similarity + weight * similarity_fine
    t2i_cmc, _ = rank(torch.tensor(final_similarity), torch.tensor(text_pid), torch.tensor(image_pid), topk, get_mAP=False)
    print(t2i_cmc)

0.0
tensor([60.6725, 79.1423, 86.4035])
0.1
tensor([60.7212, 79.0936, 86.4847])
0.2
tensor([60.7862, 79.1748, 86.5172])
0.3
tensor([60.7375, 79.3047, 86.5660])
0.4
tensor([60.6888, 79.2723, 86.7284])
0.5
tensor([60.6725, 79.4672, 86.6959])
0.6
tensor([60.6563, 79.4184, 86.6472])
0.7
tensor([60.6563, 79.3535, 86.6634])
0.8
tensor([60.7212, 79.3372, 86.6147])
0.9
tensor([60.6563, 79.4184, 86.5822])
1.0
tensor([60.7537, 79.4184, 86.6309])
1.1
tensor([60.6400, 79.4022, 86.5984])
1.2
tensor([60.5426, 79.4022, 86.5335])
1.3
tensor([60.5263, 79.3860, 86.5335])
1.4
tensor([60.4613, 79.3860, 86.5984])
1.5
tensor([60.4776, 79.3860, 86.5335])
1.6
tensor([60.4451, 79.3860, 86.5172])
1.7
tensor([60.4613, 79.4347, 86.4847])
1.8
tensor([60.4289, 79.3697, 86.4685])
1.9
tensor([60.4289, 79.3697, 86.4685])


In [18]:
t2i_cmc, _ = rank(torch.tensor(similarity_fine), torch.tensor(text_pid), torch.tensor(image_pid), topk, get_mAP=False)
print(t2i_cmc)

tensor([54.0936, 74.9513, 83.1222])
